This will be used for Coursera Capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


Business problem: 
There is a family that wants to open a family run authentic Japanese restaurant in Toronto. They are not a chain and are going for the loyal reliable customer base rather than flashy tourist attraction. They want to open a restaurant in a relatively high population density area first and somewhere where there isn’t a lot of similar competition and then where there isn’t a lot of other restaurants either considering that they think that this will increase their chances to gain customers if they aren’t spoiled for choice. They want a local supply of staples in Japanese cuisine such as fish, as these tend to go off rather quickly so ideally, they will want a nearby fresh fish market.  If there is a fresh food market in all of their top regions then amount of popular attractions will be considered as the final deciding factor.
Deciding factors:
1.	Access to local fishing docks/fish market 
2.	Population density- to target loyal clientele that will come back, not tourists
3.	Other restaurants of the same type in the area
4.	Amount of all restaurants in the area
Nice to have all else being equal:
5.	Foot traffic, where in Toronto is there the highest density of most popular attractions

Data:
1.	Foursquare data on markets containing keywords ‘fresh fish’
2.	Data from https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods
3.	Foursquare data on restaurants in the area containing keywords ‘Japanese food’
4.	Foursquare data on all restaurants in the area 
5.	Foursquare top 50 picks of locations for Toronto
